# Ejemplo de clasificación de Personalize <a class="anchor" id="top"></a>

En este cuaderno, elegirá un conjunto de datos y lo preparará para utilizarlo con las recomendaciones por lote de Amazon Personalize.

1. [Elija un conjunto de datos o un origen de datos](#source)
1. [Prepare los datos](#prepare)
1. [Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones](#group_dataset)
1. [Configurar un bucket de S3 y un rol de IAM](#bucket_role)
1. [Importar los datos de las interacciones](#import)
1. [Crear soluciones](#solutions)
1. [Crear una campaña](#create)
1. [Interactuar con campañas](#interact)
1. [Limpiar](#cleanup)

## Introducción <a class="anchor" id="intro"></a>

Por lo general, los algoritmos en Amazon Personalize (llamados recetas) buscan resolver diferentes tareas que se explican a continuación:

1. **HRNN & HRNN-Metadata**: recomienda elementos basados en las interacciones previas del usuario con los elementos.
1. **HRNN-Coldstart**: recomienda nuevos elementos para los que aún no se dispone de datos de interacción.
1. **Personalized-Ranking**: toma una recopilación de elementos y, luego, los ubica en un posible orden de interés mediante un enfoque similar al de una HRNN.
1. **SIMS (elementos similares)**: en base a un elemento, recomienda otros elementos con los que también interactúan los usuarios.
1. **Popularity-Count**: recomienda los elementos más populares, si la HRNN o los metadatos de la HRNN no ofrecen una respuesta. Esta es la devolución predeterminada.

Sin importar cuál sea el caso de uso, todos los algoritmos comparten una base de aprendizaje sobre los datos de interacción entre el usuario y los elementos, la cual se define según 3 atributos principales:

1. **UserID**: el usuario que interactuó
1. **ItemID**: el elemento con el que interactuó el usuario
1. **Marca temporal**: la hora a la que ocurrió la interacción

También admitimos los tipos de eventos y valores de eventos definidos por lo siguiente:

1. **Tipo de evento**: etiquetado de eventos por categoría (navegación, compra, clasificación, etc.).
1. **Valor de evento**: valor que corresponde al tipo de evento acontecido. En términos generales, buscamos valores normalizados entre 0 y 1 entre los tipos de eventos. Por ejemplo, si existen tres fases para completar una transacción (seleccionar, agregar al carrito y comprar), entonces hay un valor de evento para cada fase, como 0.33, 0.66 y 1.0, respectivamente.

Los campos para el tipo de evento y valor de evento son datos adicionales que se pueden utilizar para filtrar los datos enviados para el entrenamiento del modelo de personalización. En este ejercicio en particular no tendremos un tipo de evento ni un tipo de valor. 

## Elija un conjunto de datos o un origen de datos <a class="anchor" id="source"></a>
[Regresar al principio](#top)

Como ya lo mencionamos, los datos de la interacción del usuario con los elementos son cruciales para comenzar con el servicio. Esto significa que necesitamos buscar los casos de uso que generan ese tipo de datos, algunos ejemplos comunes son los siguientes:

1. Aplicaciones de video bajo demanda
1. Plataformas de comercio electrónico
1. Agregadores/plataformas de redes sociales

Existen algunas pautas para evaluar un problema que sea adecuado para Personalize. Como punto inicial, recomendamos los valores que figuran a continuación, aunque los [límites oficiales](https://docs.aws.amazon.com/personalize/latest/dg/limits.html) son un poco más bajos.

* Usuarios autenticados
* Al menos 50 usuarios exclusivos
* Al menos 100 elementos exclusivos
* Al menos 2 docenas de interacciones para cada usuario 

La mayoría de las veces esto es fácil de lograr, y si no llega a esta cantidad en alguna categoría, puede compensarlo con una mayor cantidad en otra categoría.

En general, Personalize no enviará los datos en forma perfecta y deberá realizar algunas modificaciones para que la estructura sea la correcta. Este cuaderno tiene como fin guiarle en este proceso. 

Para comenzar, utilizaremos el conjunto de datos de [Last.FM](https://grouplens.org/datasets/hetrec-2011/). Estos son registros del comportamiento de escucha musical de los usuarios. Los datos son adecuados para nuestras pautas con una gran cantidad de usuarios, elementos e interacciones.

Primero, descargará el conjunto de datos y lo extraerá en una nueva carpeta por medio del código que aparece a continuación.

In [4]:
data_dir = "data"
!mkdir $data_dir
!cd $data_dir && wget http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
!cd $data_dir && unzip hetrec2011-lastfm-2k.zip

--2020-05-26 20:45:51--  http://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2589075 (2.5M) [application/zip]
Saving to: ‘hetrec2011-lastfm-2k.zip’

hetrec2011-lastfm-2 100%[===================>]   2.47M  10.2MB/s    in 0.2s    

2020-05-26 20:45:51 (10.2 MB/s) - ‘hetrec2011-lastfm-2k.zip’ saved [2589075/2589075]

Archive:  hetrec2011-lastfm-2k.zip
  inflating: user_friends.dat        
  inflating: user_taggedartists.dat  
  inflating: user_taggedartists-timestamps.dat  
  inflating: artists.dat             
  inflating: readme.txt              
  inflating: tags.dat                
  inflating: user_artists.dat        


Observe los archivos de datos que ha descargado.

In [5]:
!ls $data_dir

artists.dat		  tags.dat	    user_taggedartists.dat
hetrec2011-lastfm-2k.zip  user_artists.dat  user_taggedartists-timestamps.dat
readme.txt		  user_friends.dat


Actualmente, no se sabe mucho acerca de los datos, excepto que parece que hay muchos archivos .dat y un archivo README. Si abrimos el archivo README, este nos dará información acerca de la estructura general de estos datos. Este es un paso que quizá pueda omitir con datos personalizados, a menos que el origen de datos provenga de un equipo externo.

A partir del archivo README, podemos ver que existen varios tipos de interacciones en este conjunto de datos. Las interacciones entre usuarios que se definen como amigos entre ellos, las interacciones de usuarios que escuchan a artistas y las interacciones de etiquetas asignadas a los usuarios y artistas.

En este caso, nos enfocamos en los usuarios, los artistas y las interacciones de escucha. Tenemos 1892 usuarios, 17 632 artistas (nuestros elementos en este caso) y 92 834 interacciones de escucha de artistas por parte de usuarios. Esto es más que suficiente para que comencemos con Personalize.

Continúe leyendo atentamente el archivo README para llegar a la sección `Files`. La mayoría de los archivos en el conjunto de datos no son relevantes para nosotros, pero el archivo `users_artists.dat` se ve prometedor. La sección `Data format` del archivo README, proporciona más detalles sobre el contenido del archivo. Aquí es donde nos encontramos con el primer problema.

| userID | artistID | ponderación  |
|--------|----------|---------|
| 2      | 51       | 13883   |

Aunque existen datos de interacción entre los usuarios y los artistas que escuchan, estas interacciones quedan guardadas como ponderaciones, en lugar de marcas temporales. Para Amazon Personalize, necesitamos los datos de marcas temporales de la interacción entre el usuario y los elementos. 

Si observa nuevamente los archivos en el conjunto de datos, debería poder ver que `users_taggedartists-timestamps.dat` contiene datos de marca temporal. Entonces, ¿qué sucede si en lugar del comportamiento de escucha, utilizamos el comportamiento de etiquetado como nuestros datos de interacción? ¿Podemos suponer que si un usuario etiqueta a un artista esto indica un sentimiento positivo? Normalmente, lo discutiría con el cliente o alguien que tenga conocimiento de dominio para comprender si esta interacción es adecuada para el caso de uso que desea resolver. Por ahora, supondremos que el comportamiento de etiquetado es adecuado para nuestras necesidades. 

El esquema para `user_taggedartists-timestamps.dat` es:

| userID | artistID | tagID | marca temporal     |
|--------|----------|-------|---------------|
| 2      | 52       | 13    | 1238536800000 |

Si eliminamos el atributo `tagID`, tenemos exactamente el formato que necesitamos para Amazon Personalize.

## Prepare los datos <a class="anchor" id="prepare"></a>
[Regresar al principio](#top)

Lo que debe hacer a continuación es cargar los datos y confirmar que los datos se encuentran en buen estado. Luego, debe guardarlos en un archivo CSV donde estén listos para su uso con Amazon Personalize.

Para comenzar, importe una recopilación de bibliotecas de Python que se utilicen normalmente en la ciencia de datos.

In [6]:
import time
from time import sleep
import json
from datetime import datetime
import numpy as np
import boto3
import pandas as pd

Luego, abra el archivo de datos y observe varias de las primeras filas.

In [7]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat')
original_data.head(5)

,userID artistID tagID timestamp
0,2\t52\t13\t1238536800000
1,2\t52\t15\t1238536800000
2,2\t52\t18\t1238536800000
3,2\t52\t21\t1238536800000
4,2\t52\t41\t1238536800000


Evidentemente, los datos no se cargaron de forma correcta. El delimitador predeterminado para los archivos CSV (valores separados por comas) es una coma (`,`), pero en este caso el archivo se guardó con delimitadores de pestaña (`\t`). Entonces, especifiquemos el delimitador correcto e intentemos cargar los datos nuevamente.

In [8]:
original_data = pd.read_csv(data_dir + '/user_taggedartists-timestamps.dat', delimiter='\t')
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1238536800000
1,2,52,15,1238536800000
2,2,52,18,1238536800000
3,2,52,21,1238536800000
4,2,52,41,1238536800000


Eso es mejor. Ahora que se han cargado los datos correctamente en la memoria, extraigamos información adicional. Primero, calculemos algunas estadísticas básicas a partir de los datos.

In [9]:
original_data.describe()

,userID,artistID,tagID,timestamp
count,186479.000000,186479.000000,186479.000000,1.864790e+05
mean,1035.600137,4375.845328,1439.582913,1.239204e+12
std,622.461272,4897.789595,2775.340279,4.299091e+10
min,2.000000,1.000000,1.000000,-4.287204e+11
25%,488.000000,686.000000,79.000000,1.209593e+12
50%,1021.000000,2203.000000,195.000000,1.243807e+12
75%,1624.000000,6714.000000,887.000000,1.275343e+12
max,2100.000000,18744.000000,12647.000000,1.304941e+12


Esto muestra que tenemos un buen rango de valores para `userID` y `artistID`. Luego, siempre es una buena idea confirmar el formato de los datos.

In [10]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 186479 entries, 0 to 186478
Data columns (total 4 columns):
userID       186479 non-null int64
artistID     186479 non-null int64
tagID        186479 non-null int64
timestamp    186479 non-null int64
dtypes: int64(4)
memory usage: 5.7 MB


En base a esto, puede ver que hay un total de 186 479 entradas en el conjunto de datos, con 4 columnas y cada celda almacenada en formato int64.

El formato int64 es claramente adecuado para `userID` y `artistID`. Sin embargo, necesitamos examinar más a fondo para comprender las marcas temporales en los datos. Para utilizar Amazon Personalize, necesita guardar las marcas temporales en el formato [Unix Epoch (época de Unix)](https://en.wikipedia.org/wiki/Unix_time).

Actualmente, los valores de las marcas temporales no son legibles para el ser humano. Así que tomemos un valor de marca temporal arbitrario y resolvamos cómo interpretarlo.

In [11]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000000


ValueError: year 41132 is out of range

¡Ups! Para este valor de marca temporal en particular, el código generó el año 41 132. Eso es un futuro algo lejano para nosotros, por lo que claramente esta no era la forma correcta de analizar los datos. Necesitamos hacer un segundo intento.

JavaScript registra el tiempo en milisegundos y esta es una recopilación de datos de una aplicación web, así que dividamos el valor de la marca temporal por 1000 antes de aplicar nuestro código.

In [12]:
arb_time_stamp = arb_time_stamp/1000
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

2009-02-28 23:00:00


Febrero de 2009 se siente mucho más realista para nuestro conjunto de datos. Aunque no necesitamos marcas temporales legibles para humanos para utilizar Amazon Personalize, sí queremos que las fechas sean realistas, así que ahora transforme cada marca temporal en un conjunto de datos alejado del formato de milisegundos de JavaScript. 

In [13]:
original_data.timestamp = original_data.timestamp / 1000
original_data.head(5)

,userID,artistID,tagID,timestamp
0,2,52,13,1.238537e+09
1,2,52,15,1.238537e+09
2,2,52,18,1.238537e+09
3,2,52,21,1.238537e+09
4,2,52,41,1.238537e+09


Haga una comprobación de validez rápida en el conjunto de datos transformado mediante la elección de una marca temporal arbitraria y su transformación a un formato legible para humanos.

In [14]:
arb_time_stamp = original_data.iloc[50]['timestamp']
print(arb_time_stamp)
print(datetime.utcfromtimestamp(arb_time_stamp).strftime('%Y-%m-%d %H:%M:%S'))

1235862000.0
2009-02-28 23:00:00


Estos datos son coherentes como una marca temporal, para que podamos continuar formateando el resto de los datos. Recuerde que los datos que necesitamos son datos de interacción entre el usuario y los elementos, que serían `userID`, `artistID` y `timestamp` en este caso. Nuestro conjunto de datos tiene una columna adicional, `tagID`, la cual se puede eliminar del conjunto de datos.

In [15]:
interactions_df = original_data.copy()
interactions_df = interactions_df[['userID', 'artistID', 'timestamp']]
interactions_df.head()

,userID,artistID,timestamp
0,2,52,1.238537e+09
1,2,52,1.238537e+09
2,2,52,1.238537e+09
3,2,52,1.238537e+09
4,2,52,1.238537e+09


Luego de manipular los datos, siempre confirme si el formato de datos ha sido modificado.

In [16]:
interactions_df.dtypes

userID         int64
artistID       int64
timestamp    float64
dtype: object

En este caso, la columna de marca temporal pasó de int64 a float64. Entonces cambiemos el formato de vuelta a int64.

In [17]:
interactions_df.astype({'timestamp': 'int64'}).dtypes

userID       int64
artistID     int64
timestamp    int64
dtype: object

 Amazon Personalize tiene nombres de columnas predeterminados para los usuarios, los elementos y las marcas temporales. Estos nombres de columnas predeterminados son `USER_ID`, `ITEM_ID` Y `TIMESTAMP`. Por lo que la última modificación del conjunto de datos es reemplazar la columna de encabezados existente por los encabezados predeterminados.

In [18]:
interactions_df.rename(columns = {'userID':'USER_ID', 'artistID':'ITEM_ID', 
                              'timestamp':'TIMESTAMP'}, inplace = True) 

¡Eso es todo! A esta altura, los datos están listos y solo necesitamos guardarlos como un archivo CSV.

In [19]:
interactions_filename = "interactions.csv"
interactions_df.to_csv((data_dir+"/"+interactions_filename), index=False, float_format='%.0f')

## Crear grupos de conjuntos de datos y el conjunto de datos de las interacciones <a class="anchor" id="group_dataset"></a>
[Regresar al principio](#top)

El nivel más alto de aislamiento y abstracción con Amazon Personalize es un *grupo de conjuntos de datos*. La información almacenada dentro de uno de estos grupos de conjunto de datos no afecta ninguno de los demás grupos de conjuntos de datos o modelos creados de uno de ellos; están completamente aislados. Esto permite ejecutar muchos experimentos y es parte de lo que hacemos para mantener a los modelos de forma privada y completamente entrenados en sus datos. 

Antes de importar los datos que preparó antes, debe haber un grupo de conjuntos de datos y un conjunto de datos agregados a este que maneje las interacciones.

Los grupos de conjuntos de datos pueden almacenar los siguientes tipos de información:

* Interacciones entre el usuario y el elemento
* Flujo de eventos (interacciones en tiempo real)
* Metadatos del usuario
* Metadatos del elemento

Antes de crear el grupo de conjuntos de datos y el conjunto de datos para nuestros datos de interacción, validemos que su entorno se pueda comunicar exitosamente con Amazon Personalize.

In [20]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')
personalize_runtime = boto3.client('personalize-runtime')

### Crear el grupo de conjuntos de datos

La siguiente celda creará un nuevo grupo de conjuntos de datos con el nombre `personalize-poc-lastfm`.

In [21]:
create_dataset_group_response = personalize.create_dataset_group(
    name = "personalize-ranking-dsg"
)

dataset_group_arn = create_dataset_group_response['datasetGroupArn']
print(json.dumps(create_dataset_group_response, indent=2))

{
  "datasetGroupArn": "arn:aws:personalize:us-east-1:144386903708:dataset-group/personalize-ranking-dsg",
  "ResponseMetadata": {
    "RequestId": "386224ac-8410-4f64-ba4a-460ba12607c7",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:46:27 GMT",
      "x-amzn-requestid": "386224ac-8410-4f64-ba4a-460ba12607c7",
      "content-length": "102",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de que podamos utilizar el grupo de conjuntos de datos, este debe estar activo. Esto puede llevar uno o dos minutos. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del grupo de conjuntos de datos a cada segundo, hasta por un máximo de 3 horas.

In [22]:
max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_group_response = personalize.describe_dataset_group(
        datasetGroupArn = dataset_group_arn
    )
    status = describe_dataset_group_response["datasetGroup"]["status"]
    print("DatasetGroup: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetGroup: CREATE PENDING
DatasetGroup: ACTIVE


Ahora que tiene un grupo de conjuntos de datos, puede crear un conjunto de datos para los datos de interacción.

### Crear el conjunto de datos

Primero, defina un esquema para indicar a Amazon Personalize el tipo de conjunto de datos que está cargando. Existen varias palabras clave reservadas y obligatorias, necesarias para el esquema, según el tipo de conjunto de datos. Para obtener más información, consulte la [documentación](https://docs.aws.amazon.com/personalize/latest/dg/how-it-works-dataset-schema.html).

Aquí, creará un esquema para datos de interacciones, para lo que se necesitan los campos `USER_ID`, `ITEM_ID` y `TIMESTAMP`. En el esquema, estos deben estar definidos en el mismo orden que aparecen el conjunto de datos.

In [23]:
interactions_schema = schema = {
    "type": "record",
    "name": "Interactions",
    "namespace": "com.amazonaws.personalize.schema",
    "fields": [
        {
            "name": "USER_ID",
            "type": "string"
        },
        {
            "name": "ITEM_ID",
            "type": "string"
        },
        {
            "name": "TIMESTAMP",
            "type": "long"
        }
    ],
    "version": "1.0"
}

create_schema_response = personalize.create_schema(
    name = "personalize-ranking-interactions",
    schema = json.dumps(interactions_schema)
)

schema_arn = create_schema_response['schemaArn']
print(json.dumps(create_schema_response, indent=2))

{
  "schemaArn": "arn:aws:personalize:us-east-1:144386903708:schema/personalize-ranking-interactions",
  "ResponseMetadata": {
    "RequestId": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:47:58 GMT",
      "x-amzn-requestid": "d3fb42c4-9a3c-45a4-a842-adf60ef03906",
      "content-length": "98",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Con un esquema creado, puede crear un conjunto de datos dentro del grupo de conjuntos de datos. Nota: Esto aún no carga los datos. Esto sucederá unos pasos más adelante.

In [24]:
dataset_type = "INTERACTIONS"
create_dataset_response = personalize.create_dataset(
    name = "personalize-ranking-ds",
    datasetType = dataset_type,
    datasetGroupArn = dataset_group_arn,
    schemaArn = schema_arn
)

interactions_dataset_arn = create_dataset_response['datasetArn']
print(json.dumps(create_dataset_response, indent=2))

{
  "datasetArn": "arn:aws:personalize:us-east-1:144386903708:dataset/personalize-ranking-dsg/INTERACTIONS",
  "ResponseMetadata": {
    "RequestId": "98a3c225-31aa-49be-936f-f633aa6314d0",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:48:02 GMT",
      "x-amzn-requestid": "98a3c225-31aa-49be-936f-f633aa6314d0",
      "content-length": "104",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


## Configurar un bucket de S3 y un rol de IAM <a class="anchor" id="bucket_role"></a>
[Regresar al principio](#top)

Hasta ahora, hemos descargado, manipulado y guardado los datos en la instancia de Amazon EBS adjunta a la instancia que ejecuta este cuaderno de Jupyter. Sin embargo, Amazon Personalize necesitará un bucket de S3 para actuar como origen de los datos, así como roles de IAM para acceder a ese bucket. Preparemos todo esto.

Utilice los metadatos almacenados en la instancia subyacente a este cuaderno de Amazon SageMaker para determinar la región en la que opera. Si utiliza un cuaderno de Jupyter fuera de Amazon SageMaker, simplemente defina la región como una cadena a continuación. El bucket de Amazon S3 debe estar en la misma región que los recursos de Amazon Personalize que hemos creado hasta ahora.

In [25]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
    data = json.load(notebook_info)
    resource_arn = data['ResourceArn']
    region = resource_arn.split(':')[3]
print(region)

us-east-1


Los nombres de los buckets de Amazon S3 son exclusivos a nivel global. Para crear un nombre de bucket exclusivo, el siguiente código agregará la cadena `personalizepoc` a su número de cuenta de AWS. Luego, crea un bucket con este nombre en la región descubierta en la celda anterior.

In [26]:
s3 = boto3.client('s3')
suffix = str(np.random.uniform())[4:9]
bucket_name = "personalize-ranking-demo-"+   suffix        # replace with the name of your S3 bucket
print(bucket_name)
if region != "us-east-1":
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
else:
    s3.create_bucket(Bucket=bucket_name)

personalize-ranking-demo-41600


### Cargar los datos a S3

Ahora que creó su bucket de Amazon S3, cargue el archivo CSV de nuestros datos de interacción entre el usuario y los elementos. 

In [27]:
interactions_file_path = data_dir + "/" + interactions_filename
boto3.Session().resource('s3').Bucket(bucket_name).Object(interactions_filename).upload_file(interactions_file_path)
interactions_s3DataPath = "s3://"+bucket_name+"/"+interactions_filename

### Establecer la política del bucket de S3
Amazon Personalize debe ser capaz de leer el contenido de su bucket de S3. Así que agregue una política de bucket que lo permita.

In [28]:
policy = {
    "Version": "2012-10-17",
    "Id": "PersonalizeS3BucketAccessPolicy",
    "Statement": [
        {
            "Sid": "PersonalizeS3BucketAccessPolicy",
            "Effect": "Allow",
            "Principal": {
                "Service": "personalize.amazonaws.com"
            },
            "Action": [
                "s3:*Object",
                "s3:ListBucket"
            ],
            "Resource": [
                "arn:aws:s3:::{}".format(bucket_name),
                "arn:aws:s3:::{}/*".format(bucket_name)
            ]
        }
    ]
}

s3.put_bucket_policy(Bucket=bucket_name, Policy=json.dumps(policy))

{'ResponseMetadata': {'RequestId': '61279AD0FB422CD2',
  'HostId': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'pvAc3tsFJjLUU5BTudF9gokRma/yRh+uH3TdrVXk3/xoLlHsOev2XPv8KDc+TusZphyzL/TDUvE=',
   'x-amz-request-id': '61279AD0FB422CD2',
   'date': 'Tue, 26 May 2020 20:48:18 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

### Crear un rol de IAM

Amazon Personalize necesita la capacidad de asumir roles en AWS de modo que tenga los permisos para ejecutar ciertas tareas. Creemos un rol de IAM y adjuntemos las políticas necesarias a este. El siguiente código adjunta políticas muy permisivas. Utilice políticas más restrictivas para cualquier aplicación en producción.

In [29]:
iam = boto3.client("iam")

role_name = "PersonalizeRoleRanking"
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "personalize.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like to use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/service-role/AmazonPersonalizeFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::144386903708:role/PersonalizeRoleRanking


## Importar los datos de las interacciones <a class="anchor" id="import"></a>
[Regresar al principio](#top)

Anteriormente, creó el grupo de conjuntos de datos y el conjunto de datos para alojar su información. Ahora ejecutará un trabajo de importación que cargará los datos del bucket de S3 en el conjunto de datos de Amazon Personalize. 

In [30]:
create_dataset_import_job_response = personalize.create_dataset_import_job(
    jobName = "personalize-ranking-example",
    datasetArn = interactions_dataset_arn,
    dataSource = {
        "dataLocation": "s3://{}/{}".format(bucket_name, interactions_filename)
    },
    roleArn = role_arn
)

dataset_import_job_arn = create_dataset_import_job_response['datasetImportJobArn']
print(json.dumps(create_dataset_import_job_response, indent=2))

{
  "datasetImportJobArn": "arn:aws:personalize:us-east-1:144386903708:dataset-import-job/personalize-ranking-example",
  "ResponseMetadata": {
    "RequestId": "7eab862d-9724-41dc-87d3-7455b4f16ead",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 20:51:57 GMT",
      "x-amzn-requestid": "7eab862d-9724-41dc-87d3-7455b4f16ead",
      "content-length": "115",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


Antes de que podamos utilizar el grupo de conjuntos de datos, el trabajo de importación debe estar activo. Ejecute la celda a continuación y aguarde hasta que muestre el estado ACTIVO. Esto verifica el estado del trabajo de importación a cada segundo, por hasta un máximo de 3 horas.

La importación de los datos puede llevar algo de tiempo, según el tamaño del conjunto de datos. En este taller, el trabajo de importación debería tomar alrededor de 15 minutos.

In [31]:
%%time

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_import_job_response = personalize.describe_dataset_import_job(
        datasetImportJobArn = dataset_import_job_arn
    )
    status = describe_dataset_import_job_response["datasetImportJob"]['status']
    print("DatasetImportJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)

DatasetImportJob: CREATE PENDING
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: CREATE IN_PROGRESS
DatasetImportJob: ACTIVE
CPU times: user 50 ms, sys: 15.1 ms, total: 65 ms
Wall time: 13min


Cuando la importación del conjunto de datos está activa, está listo para comenzar a crear modelos con SIMS, Personalized-Ranking, Popularity-Count y HRNN. Este proceso continuará en otros cuadernos. Ejecute la celda a continuación antes de pasar a almacenar algunos valores para su uso en los siguientes cuadernos.

## Crear soluciones <a class="anchor" id="solutions"></a>
[Regresar al principio](#top)

En este cuaderno, creará soluciones con las siguientes recetas:

1. HRNN


Una variación específica de un algoritmo se denomina receta en Amazon Personalize. Las diferentes recetas son adecuadas para diferentes situaciones. Un modelo entrenado se denomina solución, y cada solución puede tener muchas versiones relacionadas con un determinado volumen de datos cuando se entrenó el modelo.

Para comenzar, enumeraremos todas las recetas que son compatibles. Esto le permitirá seleccionar una y utilizarla para crear el modelo.

In [32]:
personalize.list_recipes()

{'recipes': [{'name': 'aws-hrnn',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-coldstart',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-coldstart',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-hrnn-metadata',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-hrnn-metadata',
   'status': 'ACTIVE',
   'creationDateTime': datetime.datetime(2019, 6, 10, 0, 0, tzinfo=tzlocal()),
   'lastUpdatedDateTime': datetime.datetime(2020, 5, 22, 6, 20, 26, 250000, tzinfo=tzlocal())},
  {'name': 'aws-personalized-ranking',
   'recipeArn': 'arn:aws:personalize:::recipe/aws-personalized-ranking',
   's

### Clasificación personalizada

La clasificación personalizada es una aplicación interesante de la HRNN. En lugar de limitarse a recomendar lo más probable para el usuario en cuestión, este algoritmo toma un usuario y también una lista de elementos. A continuación, los elementos se vuelven a presentar en el orden de relevancia más probable para el usuario. El caso de uso aquí es para filtrar por género, por ejemplo, o cuando se tiene una amplia colección que se desea ordenar mejor para un usuario en particular.

Para nuestro caso de uso, mediante los datos de LastFM, podríamos imaginar que una determinada discográfica nos paga por recomendar sus artistas a nuestros usuarios en una promoción especial. Por lo tanto, conocemos la lista de artistas que queremos recomendar, pero queremos averiguar cuáles de estos artistas le gustarán más a cada usuario. Utilizaríamos la clasificación personalizada para reordenar la lista de artistas de cada usuario, basándonos en su historial de etiquetado anterior. 

Al igual que la última vez, empezamos por seleccionar la receta.

In [33]:
rerank_recipe_arn = "arn:aws:personalize:::recipe/aws-personalized-ranking"

#### Crear la solución

Al igual que con la solución anterior, comience por crear la solución primero. Aunque en este paso se proporciona el conjunto de datos ARN, el modelo aún no está entrenado. Véalo como un identificador en lugar de un modelo entrenado.

In [34]:
rerank_create_solution_response = personalize.create_solution(
    name = "personalize-ranking",
    datasetGroupArn = dataset_group_arn,
    recipeArn = rerank_recipe_arn
)

rerank_solution_arn = rerank_create_solution_response['solutionArn']
print(json.dumps(rerank_create_solution_response, indent=2))

{
  "solutionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking",
  "ResponseMetadata": {
    "RequestId": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "ba5cdcc0-de91-4bf0-a06d-dde91ff1b6ac",
      "content-length": "89",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


#### Crear la versión de la solución

Una vez que tenga una solución, deberá crear una versión a fin de completar el entrenamiento del modelo. Completar el entrenamiento puede llevar un tiempo, más de 25 minutos, y un promedio de 35 minutos para esta receta con nuestro conjunto de datos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas, y el objetivo aquí es crear muchos modelos e implementarlos rápidamente. Así que estableceremos el bucle while para todas las soluciones más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [35]:
rerank_create_solution_version_response = personalize.create_solution_version(
    solutionArn = rerank_solution_arn
)

In [36]:
rerank_solution_version_arn = rerank_create_solution_version_response['solutionVersionArn']
print(json.dumps(rerank_create_solution_version_response, indent=2))

{
  "solutionVersionArn": "arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819",
  "ResponseMetadata": {
    "RequestId": "b07c347c-48aa-4650-86f6-53197351a62f",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:04:58 GMT",
      "x-amzn-requestid": "b07c347c-48aa-4650-86f6-53197351a62f",
      "content-length": "105",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Ver el estado de creación de la solución

Según lo prometido, veremos cómo se pueden observar las actualizaciones de estado en la consola:

* En otra pestaña del navegador, ya debería tener la consola de AWS activa desde la apertura de esta instancia de cuaderno. 
* Cambie a esa pestaña y busque en la parte superior el servicio `Personalize`, luego diríjase a la página de ese servicio. 
* Haga clic en `View dataset groups` (Ver grupos de conjuntos de datos).
* Haga clic en el nombre de su grupo de conjuntos de datos, probablemente contenga POC en el nombre.
* Haga clic en `Solutions and recipes` (Ver grupos de conjuntos de datos).
* Ahora verá una lista de todas las soluciones que creó anteriormente, incluida una columna con el estado de las versiones de la solución. Una vez que esté `Active` (ACTIVA), su solución estará lista para la revisión. También es capaz de implementarse.

O simplemente ejecute la celda a continuación para seguir el estado de creación de la versión de la solución.

In [38]:
in_progress_solution_versions = [
    rerank_solution_version_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for solution_version_arn in in_progress_solution_versions:
        version_response = personalize.describe_solution_version(
            solutionVersionArn = solution_version_arn
        )
        status = version_response["solutionVersion"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(solution_version_arn))
            in_progress_solution_versions.remove(solution_version_arn)
    
    if len(in_progress_solution_versions) <= 0:
        break
    else:
        print("At least one solution build is still in progress")
        
    time.sleep(60)

Build succeeded for arn:aws:personalize:us-east-1:144386903708:solution/personalize-ranking/0f00a819


## Crear una campaña <a class="anchor" id="create"></a>
[Regresar al principio](#top)

Una campaña es una versión de la solución alojada; un punto de conexión que puede consultar para obtener recomendaciones. El precio se fija estimando la capacidad de rendimiento (solicitudes de personalización de los usuarios por segundo). Cuando implementa una campaña, se establece un valor mínimo de rendimiento por segundo (TPS). Este servicio, como muchos dentro de AWS, se escalará automáticamente en función de la demanda, pero si la latencia es crítica, es posible que desee aprovisionar por adelantado para una mayor demanda. Para esta POC y la demostración, todos los límites mínimos de rendimiento se establecen en 1. Para obtener más información, consulte la [página de precios](https://aws.amazon.com/personalize/pricing/).

Empecemos a implementar las campañas.

### Clasificación personalizada

Implemente una campaña para su versión de la solución de clasificación personalizada. La implementación de una campaña puede tardar unos 10 minutos. Por lo general, utilizaríamos un bucle while para hacer un seguimiento hasta que se complete la tarea. Sin embargo, la tarea bloquearía la ejecución de otras celdas y el objetivo aquí es crear múltiples campañas. Así que estableceremos el bucle while para todas las campañas más adelante en el cuaderno. Allí, también encontrará instrucciones para ver el progreso en la consola de AWS.

In [39]:
rerank_create_campaign_response = personalize.create_campaign(
    name = "personalize-poc-rerank",
    solutionVersionArn = rerank_solution_version_arn,
    minProvisionedTPS = 1
)

rerank_campaign_arn = rerank_create_campaign_response['campaignArn']
print(json.dumps(rerank_create_campaign_response, indent=2))

{
  "campaignArn": "arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank",
  "ResponseMetadata": {
    "RequestId": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
    "HTTPStatusCode": 200,
    "HTTPHeaders": {
      "content-type": "application/x-amz-json-1.1",
      "date": "Tue, 26 May 2020 21:48:55 GMT",
      "x-amzn-requestid": "61fd83dd-e409-4433-a2b9-667aecdecb0a",
      "content-length": "92",
      "connection": "keep-alive"
    },
    "RetryAttempts": 0
  }
}


### Ver el estado de creación de la campaña

Según lo prometido, veremos cómo se pueden observar las actualizaciones de estado en la consola:

* En otra pestaña del navegador, ya debería tener la consola de AWS activa desde la apertura de esta instancia de cuaderno. 
* Cambie a esa pestaña y busque en la parte superior el servicio `Personalize`, luego diríjase a la página de ese servicio. 
* Haga clic en `View dataset groups` (Ver grupos de conjuntos de datos).
* Haga clic en el nombre de su grupo de conjuntos de datos, probablemente contenga POC en el nombre.
* Haga clic en `Campaigns` (Ver grupos de conjuntos de datos).
* Ahora verá una lista de todas las campañas que creó anteriormente, incluida una columna con el estado de las versiones de la solución. Una vez que esté `Active`, su campaña estará lista para ser consultada.

O simplemente ejecute la celda de abajo para seguir el estado de creación de la versión de la campaña.

In [41]:
in_progress_campaigns = [
    rerank_campaign_arn
]

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    for campaign_arn in in_progress_campaigns:
        version_response = personalize.describe_campaign(
            campaignArn = campaign_arn
        )
        status = version_response["campaign"]["status"]
        
        if status == "ACTIVE":
            print("Build succeeded for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
        elif status == "CREATE FAILED":
            print("Build failed for {}".format(campaign_arn))
            in_progress_campaigns.remove(campaign_arn)
    
    if len(in_progress_campaigns) <= 0:
        break
    else:
        print("At least one campaign build is still in progress")
        
    time.sleep(60)

At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
At least one campaign build is still in progress
Build succeeded for arn:aws:personalize:us-east-1:144386903708:campaign/personalize-poc-rerank


## Interactuar con campañas <a class="anchor" id="interact"></a>
[Regresar al principio](#top)

Ahora que todas las campañas se han implementado, podemos comenzar a recibir recomendaciones a través de una llamada de API. Cada una de las campañas se basa en una receta diferente, que se comporta de forma ligeramente distinta porque sirve para diferentes casos de uso. Cubriremos cada campaña en un orden diferente al que se utilizó en cuadernos anteriores, para tratar las posibles complejidades en orden ascendente (es decir, lo más sencillo primero).

En primer lugar, crearemos una función de apoyo para ayudar a dar sentido a los resultados devueltos por una campaña de personalización. Personalize solo devuelve un `item_id`. Esto es genial para mantener los datos de forma compacta, pero supone la necesidad de consultar una base de datos o una tabla de búsqueda para obtener un resultado legible para los cuadernos. Crearemos una función de ayuda para devolver un resultado legible para los humanos a partir del conjunto de datos LastFM.

Comience descargando el conjunto de datos que podemos utilizar para nuestra tabla de búsqueda.

In [42]:
# Create a dataframe for the items by reading in the correct source CSV
items_df = pd.read_csv(data_dir + '/artists.dat', delimiter='\t', index_col=0)

# Render some sample data
items_df.head(5)

,name,url,pictureURL
id,,,
1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


Cuando definimos la columna ID como una columna índice, es común que devuelva un artista con tan solo buscar el ID.

In [43]:
item_id_example = 987
artist = items_df.loc[item_id_example]['name']
print(artist)

Earth, Wind & Fire


Eso no es malo, pero se podría volver complicado tener que repetir esto en cada parte de nuestro código, por lo que la siguiente función facilitará esto.

In [44]:
def get_artist_by_id(artist_id, artist_df=items_df):
    """
    This takes in an artist_id from Personalize so it will be a string,
    converts it to an int, and then does a lookup in a default or specified
    dataframe.
    
    A really broad try/except clause was added in case anything goes wrong.
    
    Feel free to add more debugging or filtering here to improve results if
    you hit an error.
    """
    try:
        return artist_df.loc[int(artist_id)]['name']
    except:
        return "Error obtaining artist"

Ahora probemos con algunos valores simples para verificar nuestra detección de errores.

In [45]:
# A known good id
print(get_artist_by_id(artist_id="987"))
# A bad type of value
print(get_artist_by_id(artist_id="987.9393939"))
# Really bad values
print(get_artist_by_id(artist_id="Steve"))

Earth, Wind & Fire
Error obtaining artist
Error obtaining artist


¡Genial! Ahora tenemos una forma de conseguir resultados. 

In [47]:
users_df = pd.read_csv(data_dir + '/user_artists.dat', delimiter='\t', index_col=0)
# Render some sample data
users_df.head(5)

,artistID,weight
userID,,
2,51,13883
2,52,11690
2,53,11351
2,54,10300
2,55,8983


### Clasificación personalizada

El caso de uso principal de la clasificación personalizada es tomar una colección de elementos y presentarlos en orden de prioridad o de interés probable para un usuario. Para demostrarlo, necesitaremos un usuario aleatorio y una colección aleatoria de 25 elementos.

In [48]:
rerank_user = users_df.sample(1).index.tolist()[0]
rerank_items = items_df.sample(25).index.tolist()

Ahora, cree un marco de datos agradable que muestre los datos de entrada.

In [49]:
rerank_list = []
for item in rerank_items:
    artist = get_artist_by_id(item)
    rerank_list.append(artist)
rerank_df = pd.DataFrame(rerank_list, columns = [rerank_user])
rerank_df

,1303
0,Priestess
1,Пелагея
2,Cosmic Gate
3,Mujuice
4,Prism
5,WIZO
6,Love of Lesbian
7,Mayhem
8,Taking Dawn
9,Universum


Luego, haga la llamada a la API de clasificación personalizada.

In [50]:
# Convert user to string:
user_id = str(rerank_user)
rerank_item_list = []
for item in rerank_items:
    rerank_item_list.append(str(item))
    
# Get recommended reranking
get_recommendations_response_rerank = personalize_runtime.get_personalized_ranking(
        campaignArn = rerank_campaign_arn,
        userId = user_id,
        inputList = rerank_item_list
)

get_recommendations_response_rerank

{'ResponseMetadata': {'RequestId': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/json',
   'date': 'Tue, 26 May 2020 22:02:55 GMT',
   'x-amzn-requestid': 'fd2ea40f-11f5-461c-b678-6a6a08bdb416',
   'content-length': '1442',
   'connection': 'keep-alive'},
  'RetryAttempts': 0},
 'personalizedRanking': [{'itemId': '1047', 'score': 0.1730128},
  {'itemId': '1046', 'score': 0.1639514},
  {'itemId': '1254', 'score': 0.1101557},
  {'itemId': '18692', 'score': 0.0762716},
  {'itemId': '5783', 'score': 0.0678702},
  {'itemId': '13434', 'score': 0.067667},
  {'itemId': '8915', 'score': 0.0616803},
  {'itemId': '1305', 'score': 0.0322442},
  {'itemId': '8543', 'score': 0.0297265},
  {'itemId': '11979', 'score': 0.0286533},
  {'itemId': '6820', 'score': 0.025196},
  {'itemId': '4154', 'score': 0.0246835},
  {'itemId': '2312', 'score': 0.0228279},
  {'itemId': '2158', 'score': 0.0179717},
  {'itemId': '2036', 'score': 0.0176177},
 

Ahora, agregue los elementos reordenados como una segunda columna al marco de datos original, para una comparación en paralelo.

In [51]:
ranked_list = []
item_list = get_recommendations_response_rerank['personalizedRanking']
for item in item_list:
    artist = get_artist_by_id(item['itemId'])
    ranked_list.append(artist)
ranked_df = pd.DataFrame(ranked_list, columns = ['Re-Ranked'])
rerank_df = pd.concat([rerank_df, ranked_df], axis=1)
rerank_df

,1303,Re-Ranked
0,Priestess,McFly
1,Пелагея,New Found Glory
2,Cosmic Gate,Mayhem
3,Mujuice,Farid Farjad
4,Prism,Priestess
5,WIZO,Пелагея
6,Love of Lesbian,How to Destroy Angels
7,Mayhem,Cosmic Gate
8,Taking Dawn,Prism
9,Universum,Murder by Death


Puede ver anteriormente cómo se reordenó cada entrada en función de la comprensión del usuario por parte del modelo. Esta es una tarea frecuente cuando se tiene una colección de elementos para mostrar a un usuario, una lista de promociones por ejemplo, o si se está filtrando en una categoría y se quiere mostrar los elementos más recomendables.

## Limpiar campañas

Comience por limpiar las campañas. Es necesario eliminar las campañas antes de poder eliminar las versiones de la solución en las que se basan. 

El código que aparece a continuación enumerará todos los esquemas en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_campaigns')
for paginate_result in paginator.paginate():
    for campaign in paginate_result["campaigns"]:
        print(campaign["campaignArn"])

Revise la lista de los ARN para determinar qué conjunto de datos desea eliminar. Luego, utilice el código que aparece a continuación para eliminar la campaña insertando el ARN.

In [ ]:
personalize.delete_campaign(
    campaignArn = "INSERT ARN HERE"
)

## Limpiar soluciones

Luego, limpie las soluciones. El código que aparece a continuación enumerará todas las soluciones en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_solutions')
for paginate_result in paginator.paginate():
    for solution in paginate_result["solutions"]:
        print(solution["solutionArn"])

Revise la lista de los ARN para determinar cuál solución desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_solution(
    solutionArn = "INSERT ARN HERE"
)

## Limpiar conjuntos de datos

Luego, limpie los conjuntos de datos. El código que aparece a continuación enumerará todos los conjuntos de datos en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_datasets')
for paginate_result in paginator.paginate():
    for datasets in paginate_result["datasets"]:
        print(datasets["datasetArn"])

Revise la lista de los ARN para determinar qué conjunto de datos desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_dataset(
    datasetArn = "INSERT ARN HERE"
)

## Limpiar esquemas

Luego, limpie los esquemas. El código que aparece a continuación enumerará todos los esquemas en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_schemas')
for paginate_result in paginator.paginate():
    for schema in paginate_result["schemas"]:
        print(schema["schemaArn"])

Revise la lista de los ARN para determinar qué esquema desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_schema(
    schemaArn = "INSERT ARN HERE"
)

## Limpiar grupos de conjuntos de datos

Por último, limpie el grupo de conjuntos de datos. El código que aparece a continuación enumerará todos los grupos de conjuntos de datos en la cuenta.

In [ ]:
paginator = personalize.get_paginator('list_dataset_groups')
for paginate_result in paginator.paginate():
    for dataset_group in paginate_result["datasetGroups"]:
        print(dataset_group["datasetGroupArn"])

Revise la lista de los ARN para determinar qué grupo de conjuntos de datos desea eliminar. Luego, utilice el código que aparece a continuación para eliminarlo insertando el ARN.

In [ ]:
personalize.delete_dataset_group(
    datasetGroupArn = "INSERT ARN HERE"
)

## Limpiar bucket de S3 y rol de IAM

De forma opcional, puede eliminar el rol de IAM y el bucket de S3 que creamos a lo largo del curso.

Comience por enumerar todos los roles de IAM en la cuenta con el código que aparece a continuación.

In [ ]:
iam = boto3.client('iam')

paginator = iam.get_paginator('list_roles')
for paginate_result in paginator.paginate():
    for roles in paginate_result["Roles"]:
        print(roles["RoleName"])

Identifique el nombre del rol que desea eliminar.

No puede eliminar un rol de IAM que aún tiene políticas adjuntas. Después de que identifique el rol relevante, enumere las políticas adjuntas a ese rol.

In [ ]:
iam.list_attached_role_policies(
    RoleName = "INSERT ROLE NAME HERE"
)

Debe desconectar las políticas en el resultado anterior mediante el código que aparece a continuación. Repita este paso para cada política adjunta.

In [ ]:
iam.detach_role_policy(
    RoleName = "INSERT ROLE NAME HERE",
    PolicyArn = "INSERT ARN HERE"
)

Por último, debería poder eliminar el rol de IAM.

In [ ]:
iam.delete_role(
    RoleName = "INSERT ROLE NAME HERE"
)

Para eliminar un bucket de S3, primero debe estar vacío. La forma más sencilla de eliminar un bucket de S3 es simplemente navegar a S3 en la consola de AWS, eliminar los objetos en el bucket y, luego, eliminar el bucket de S3.